# Latent Dirichlet Allocation (LDA) with Gibbs Sampling for Topic Modeling
Following the guidelines of the technical report [*A Theoretical and Practical Implementation
Tutorial on Topic Modeling and Gibbs Sampling*](https://coli-saar.github.io/cl19/materials/darling-lda.pdf).

In [2]:
import numpy as np 
import spacy
import random
import pandas as pd 
from tqdm import tqdm 
from collections import Counter

In [20]:
df = pd.read_csv('../data/emails.csv')
df.head(5)

,file,message
0,allen-p/_sent_mail/1.,Message-ID: <18782981.1075855378110.JavaMail.e...
1,allen-p/_sent_mail/10.,Message-ID: <15464986.1075855378456.JavaMail.e...
2,allen-p/_sent_mail/100.,Message-ID: <24216240.1075855687451.JavaMail.e...
3,allen-p/_sent_mail/1000.,Message-ID: <13505866.1075863688222.JavaMail.e...
4,allen-p/_sent_mail/1001.,Message-ID: <30922949.1075863688243.JavaMail.e...


In [4]:
# Dirichlet, hyperparams, tokenizer
ALPHA = 0.1
BETA = 0.1
NUM_TOPICS = 20

sp = spacy.load('en_core_web_sm') # tokenizer

# Seed for reproducibility
np.random.seed(42)
random.seed(42)

In [23]:
def generate_frequencies(data, max_docs=10000):
    freqs = Counter()
    all_stopwords = sp.Defaults.stop_words
    all_stopwords.add('enron')
    nr_tokens = 0

    for doc in data[:max_docs]:
        tokens = sp.tokenizer(doc)
        for token in tokens:
            if token.text.lower() not in all_stopwords and token.is_alpha:
                freqs[token.text.lower()] += 1
                nr_tokens += 1
    
    return freqs

def get_vocab(freqs, freq_threshold=3):
    vocab = {}
    vocab_idx_str = {}
    vocab_idx = 0

    for word in freqs:
        if freqs[word] >= freq_threshold:
            vocab[word] = vocab_idx
            vocab_idx_str[vocab_idx] = word
            vocab_idx += 1

    return vocab, vocab_idx_str

def tokenize_dataset(data, vocab, max_docs=10000):
    nr_tokens = 0
    nr_docs = 0
    docs = []

    for doc in data[:max_docs]:
        tokens = sp.tokenizer(doc)

        if len(tokens) > 1:
            doc = []
            for token in tokens:
                token_text = token.text.lower()
                if token_text in vocab:
                    doc.append(token_text)
                    nr_tokens += 1
            nr_docs += 1
            docs.append(doc)
        
    print(f"Number of emails: {nr_docs}, Number of tokens: {nr_tokens}")

    # Numericalize the dataset
    corpus = []
    for doc in docs:
        corpus_d = []
        
        for token in doc:
            corpus_d.append(vocab[token])
        
        corpus.append(np.asarray(corpus_d))

    return docs, corpus

In [33]:
data = df['message'].sample(frac=1, random_state=42).values
freqs = generate_frequencies(data, max_docs=10000)
vocab, vocab_idx_str = get_vocab(freqs)
docs, corpus = tokenize_dataset(data, vocab)
vocab_size = len(vocab)
print(f"Vocab size: {vocab_size}")

Number of emails: 10000, Number of tokens: 1992931
Vocab size: 27877


In [34]:
def LDA_Collapsed_Gibbs(corpus, num_iter=200):
    # Initialize the counts and Z
    Z = []
    num_docs = len(corpus)
    for _, doc in enumerate(corpus):
        Z_d = np.random.randint(low=0, high=NUM_TOPICS, size=len(doc))
        Z.append(Z_d)
    
    ndk = np.zeros((num_docs, NUM_TOPICS))
    for d in range(num_docs):
        for k in range(NUM_TOPICS):
            ndk[d, k] = np.sum(Z[d] == k)
    
    nkw = np.zeros((NUM_TOPICS, vocab_size))
    for doc_idx, doc in enumerate(corpus):
        for word_idx, word in enumerate(doc):
            topic = Z[doc_idx][word_idx]
            nkw[topic, word] += 1
    
    nk = np.sum(nkw, axis=1)
    topic_list = [i for i in range(NUM_TOPICS)]

    # Loop
    for _ in tqdm(range(num_iter)):
        for doc_idx, doc in enumerate(corpus):
            for i in range(len(doc)):
                word = doc[i]
                topic = Z[doc_idx][i]

                # Remove Z_i because conditioned on Z_(-i)
                ndk[doc_idx, topic] -= 1
                nkw[topic, word] -= 1
                nk[topic] -= 1

                p_z = (ndk[doc_idx, :] + ALPHA) * (nkw[:, word] + BETA) / (nk + BETA * vocab_size)
                topic = random.choices(topic_list, weights=p_z, k=1)[0]

                # Update n parameters
                Z[doc_idx][i] = topic
                ndk[doc_idx, topic] += 1
                nkw[topic, word] += 1
                nk[topic] += 1
    
    return Z, ndk, nkw, nk

Z, ndk, nkw, nk = LDA_Collapsed_Gibbs(corpus)


100%|██████████| 200/200 [31:25<00:00,  9.43s/it]


In [35]:
phi = nkw / nk.reshape(NUM_TOPICS, 1) # to get the probability distribution

num_words = 10

for k in range(NUM_TOPICS):
    most_common_words = np.argsort(phi[k])[::-1][:num_words]
    print(f"Topic {k} most common words: ")
    
    for word_idx in most_common_words:
        print(vocab_idx_str[word_idx])
    
    print("\n")

Topic 0 most common words: 
x
subject
cc
content
kaminski
vince
j
pm
meeting
bcc


Topic 1 most common words: 
power
energy
state
california
said
electricity
davis
gas
utilities
prices


Topic 2 most common words: 
x
hou
cc
subject
jones
sara
shackleton
mark
tana
legal


Topic 3 most common words: 
x
subject
cc
tw
fossum
content
scott
drew
blair
sent


Topic 4 most common words: 
pm
scheduled
sat
x
access
outages
time
system
london
fri


Topic 5 most common words: 
gas
deal
x
price
subject
day
market
deals
cc
month


Topic 6 most common words: 
x
jeff
dasovich
na
kean
j
richard
james
symes
steffes


Topic 7 most common words: 
cn
recipients
ou
na
notesaddr
non
john
x
eu
david


Topic 8 most common words: 
image
x
click
free
email
content
e
new
web
information


Topic 9 most common words: 
need
issues
time
market
information
business
work
process
group
risk


Topic 10 most common words: 
company
said
stock
new
billion
million
companies
financial
dow
year


Topic 11 most common words: 
k